In [19]:
# basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# sklearn part
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler 
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

# other models
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# other
from scipy.spatial.distance import cdist

In [20]:
SEED = 42

# Preprocessing functions

In [21]:
def fix_lat_lon(df):
    df['lat_processed'] = df.lat * 11.112
    df['lon_processed'] = df.lon * 6.4757
    return df

def fix_outliers(df):
    return df[df.revenue != 0]

# Features engineering

In [22]:
def fix_year(df):
    return df[df.year == 2016]


def create_population_repartition(df_grunnkrets_house_pers):
    tmp = df_grunnkrets_house_pers[[
            'grunnkrets_id',
            'couple_children_0_to_5_years',
            'couple_children_6_to_17_years',
            'couple_children_18_or_above',
            'couple_without_children',
            'single_parent_children_0_to_5_years',
            'single_parent_children_6_to_17_years',
            'single_parent_children_18_or_above',
            'singles'
        ]]
    return tmp.rename({column: 'hp_' + column for column in tmp.columns if column != 'grunnkrets_id'}, axis=1)


def create_income_repartition(df_grunnkrets_income_house):
    tmp = df_grunnkrets_income_house[[
            'grunnkrets_id',
            'all_households',
            'singles',
            'couple_without_children',
            'couple_with_children',
            'other_households',
            'single_parent_with_children'
            ]]
    return tmp.rename({column: 'ih_' + column for column in tmp.columns if column != 'grunnkrets_id'}, axis=1)


def create_grunnkret_geodata(df_grunnkrets_stripped):
    return df_grunnkrets_stripped[[
                            'grunnkrets_id',
                            'area_km2',
                            'municipality_name'
                            ]]


def create_hierarchy(df_plaace_hierarchy):
    return df_plaace_hierarchy[[
                            'plaace_hierarchy_id',
                            'lv1_desc',
                            'lv2_desc',
                            'lv3_desc'
                            ]]


def create_population_age(df_grunnkrets_age_dist):
    df_grunnkrets_population = df_grunnkrets_age_dist.loc[:, ['grunnkrets_id']]
    df_grunnkrets_population['total_population'] = df_grunnkrets_age_dist.iloc[:,2:92].sum(axis=1)
    df_grunnkrets_population['youngs'] = df_grunnkrets_age_dist.iloc[:, 2:20].sum(axis=1)
    df_grunnkrets_population['adults'] = df_grunnkrets_age_dist.iloc[:, 21:64].sum(axis=1)
    df_grunnkrets_population['seniors'] = df_grunnkrets_age_dist.iloc[:, 65:92].sum(axis=1)
    return df_grunnkrets_population


def find_closest_bus_stop(df, df_bus_stops):
    """
    Combine the training data with the bus stop data by finding :
    - the closest bus stop from the store (create a feature the minimal distance then)
    - the mean distance of every bus stop in 1km radius
    for each category of bus stop
    """
    df['lat_processed'] = df.lat_processed * 10
    df['lon_processed'] = df.lon_processed * 10

    categories = ['Mangler viktighetsnivå',
                  'Standard holdeplass',
                  'Lokalt knutepunkt',
                  'Nasjonalt knutepunkt',
                  'Regionalt knutepunkt',
                  'Annen viktig holdeplass']

    new_bs_features = pd.DataFrame(df.store_id)

    df_bus_tmp = df_bus_stops.loc[:, ['busstop_id']]
    df_bus_tmp[['lon_processed', 'lat_processed']] = df_bus_stops['geometry'].str.extract(
        r'(?P<lat>[0-9]*[.]?[0-9]+)\s(?P<lon>[0-9]*[.]?[0-9]+)', expand=True)
    df_bus_tmp['lon_processed'] = pd.to_numeric(df_bus_tmp['lon_processed']) * 6.4757 * 10  # value in km
    df_bus_tmp['lat_processed'] = pd.to_numeric(df_bus_tmp['lat_processed']) * 11.112 * 10  # value in km

    mat = cdist(df_bus_tmp[['lat_processed', 'lon_processed']], df[['lat_processed', 'lon_processed']], metric='euclidean')
    correlation_dist = pd.DataFrame(mat, index=df_bus_tmp['busstop_id'], columns=df['store_id'])
    new_bs_features = pd.merge(new_bs_features,
                               pd.DataFrame(correlation_dist.min(),
                                            columns=['BS_closest']),
                               on='store_id', how='left')
    new_bs_features = pd.merge(new_bs_features,
                               pd.DataFrame(correlation_dist[correlation_dist < 1].mean(),
                                            columns=['BS_mean_1km']),
                               on='store_id', how='left')
    new_bs_features = pd.merge(new_bs_features,
                               pd.DataFrame(correlation_dist[correlation_dist < 0.5].count(),
                                                columns=['number_BS_500m']), on='store_id')

    for category in categories:
        df_bus_tmp = df_bus_stops[df_bus_stops['importance_level'] == category].loc[:, ['busstop_id']]
        df_bus_tmp[['lon_processed', 'lat_processed']] = df_bus_stops['geometry'].str.extract(
                                                                r'(?P<lat>[0-9]*[.]?[0-9]+)\s(?P<lon>[0-9]*[.]?[0-9]+)',
                                                                expand=True)
        df_bus_tmp['lon_processed'] = pd.to_numeric(df_bus_tmp['lon_processed']) * 6.4757 * 10    # value in km
        df_bus_tmp['lat_processed'] = pd.to_numeric(df_bus_tmp['lat_processed']) * 11.112 * 10    # value in km

        mat = cdist(df_bus_tmp[['lat_processed', 'lat_processed']], df[['lat_processed', 'lon_processed']], metric='euclidean')
        correlation_dist = pd.DataFrame(mat, index=df_bus_tmp['busstop_id'], columns=df['store_id'])
        new_bs_features = pd.merge(new_bs_features,
                                   pd.DataFrame(correlation_dist.min(),
                                                columns=['BS_closest_' + category.lower().replace(' ', '_')]),
                                   on='store_id', how='left')
        new_bs_features = pd.merge(new_bs_features,
                                   pd.DataFrame(correlation_dist[correlation_dist < 1].mean(),
                                                columns=['BS_mean_1km_'+category.lower().replace(' ', '_')]),
                                   on='store_id', how='left')
    return new_bs_features.fillna(0)


def fix_municipalities(current_df):
    # Get the rows with missing municipality
    df_missing_mun = current_df[current_df["municipality_name"].isna()]
    # Create a copy of the current df and drop row where mun = NaN + Reset index
    current_df_copy = current_df.copy().dropna(subset=['municipality_name'])
    current_df_copy = current_df_copy.reset_index(drop=True)
    # For each missing municipality
    for index, row in df_missing_mun.iterrows():
        # Create a df with the the difference with the loc of the current store and all the others stores
        tmp_df = pd.concat([current_df_copy.loc[:, ["lat_processed"]] - row.lat_processed,
                            current_df_copy.loc[:, ["lon_processed"]] - row.lon_processed], axis=1)
        # Find the idx of the one with the smallest error (the closest from the other)
        idx = np.argmin(np.linalg.norm(tmp_df.to_numpy(), axis=1))
        # Retrieve the municipality of the closest one and input it in the missing one
        current_df.loc[index, "municipality_name"] = current_df_copy.loc[idx, "municipality_name"]
    return current_df


def keep_only_use_features(current_df, features):
    return current_df.loc[:, features]


def drop_non_use_features(current_df):
    return current_df.drop([
                        'year',
                        'store_id',
                        'store_name',
                        'address',
                        'sales_channel_name',
                        'chain_name',
                        'mall_name',
                        'plaace_hierarchy_id',
                        'lv1_desc',
                        'lv2_desc',
                        'lv3_desc',
                        'municipality_name',
                        ], axis=1)


def label_uniformier(array_train, array_test):
    """
    Take the unique values from the train and test part to combine it in a single array.
    Useful to fit the label encoder and don't do a mess during the transform (previously fit_transform that was confusing)
    """
    label_encoder = LabelEncoder()
    labels = np.asarray(list(array_train.unique()) + list(set(array_test.unique()) - set(array_train.unique())))
    label_encoder.fit(labels)
    return label_encoder


def encode_feature(df_train, df_test, feature_name):
    le = label_uniformier(df_train[feature_name], df_test[feature_name])
    df_train['encoded_' + feature_name] = le.transform(df_train[feature_name])
    df_test['encoded_' + feature_name] = le.transform(df_test[feature_name])
    return df_train, df_test


def fill_nan_mean(current_df):
    return current_df.apply(lambda x: x.fillna(x.mean()), axis=0)


def store_secret_feature(df):
    tmp_df = df.loc[:, ['store_id', ]]
    tmp_df[['SI_p1', 'SI_p2', 'SI_p3']] = df['store_id'].str.extract(r'(?P<p1>[0-9]+)-+(?P<p2>[0-9]+)-+(?P<p3>[0-9]+)', expand=True)
    tmp_df['SI_all'] = tmp_df[['SI_p1', 'SI_p2', 'SI_p3']].stack().groupby(level=0).apply(''.join)
    tmp_df[['SI_p1', 'SI_p2', 'SI_p3']] = tmp_df[['SI_p1', 'SI_p2', 'SI_p3']].apply(pd.to_numeric)
    tmp_df['SI_all'] = tmp_df['SI_all'].astype('float')
    return tmp_df


def lat_lon_precisionless(df):
    tmp_df = df.loc[:, ['store_id', 'lat', 'lon']]
    tmp_df[['lat', 'lon']] = tmp_df[['lat', 'lon']]
    tmp_df[['lat_reduced', 'lon_reduced']] = tmp_df[['lat', 'lon']].astype('int')
    return tmp_df[['store_id', 'lat_reduced', 'lon_reduced']]


def scale_values(df_train, df_test, Y_train):
    scaler = StandardScaler()
    df_train[df_train.columns] = scaler.fit_transform(df_train)
    df_test[df_train.columns] = scaler.transform(df_test)

    Y_train = np.log10(Y_train + 1)

    return df_train, df_test, Y_train


def extract_revenue(df_train):
    return df_train.loc[:, ['revenue', ]], df_train.drop(['revenue'], axis=1)


def features_engineering(df_train, df_test):
    df_bus_stops = pd.read_csv("data/busstops_norway.csv")
    df_grunnkrets_age_dist = pd.read_csv("data/grunnkrets_age_distribution.csv")
    df_grunnkrets_house_pers = pd.read_csv("data/grunnkrets_households_num_persons.csv")
    df_grunnkrets_income_house = pd.read_csv("data/grunnkrets_income_households.csv")
    df_grunnkrets_stripped = pd.read_csv("data/grunnkrets_norway_stripped.csv")
    df_plaace_hierarchy = pd.read_csv("data/plaace_hierarchy.csv")

    df_grunnkrets_stripped = fix_year(df_grunnkrets_stripped)
    df_grunnkrets_age_dist = fix_year(df_grunnkrets_age_dist)
    df_grunnkrets_house_pers = fix_year(df_grunnkrets_house_pers)
    df_grunnkrets_income_house = fix_year(df_grunnkrets_income_house)

    df_train = pd.merge(df_train, store_secret_feature(df_train), how="left", on="store_id")
    df_test = pd.merge(df_test, store_secret_feature(df_test), how="left", on="store_id")

    df_train = pd.merge(df_train, lat_lon_precisionless(df_train), how="left", on="store_id")
    df_test = pd.merge(df_test, lat_lon_precisionless(df_test), how="left", on="store_id")
    df_train['latxlat'] = df_train['lat_reduced']*df_train['lon_reduced']
    df_test['latxlat'] = df_test['lat_reduced']*df_test['lon_reduced']

    df_train = pd.merge(df_train, create_population_repartition(df_grunnkrets_house_pers), how="left", on="grunnkrets_id")
    df_test = pd.merge(df_test, create_population_repartition(df_grunnkrets_house_pers), how="left", on="grunnkrets_id")

    df_train = pd.merge(df_train, create_income_repartition(df_grunnkrets_income_house), how="left",on="grunnkrets_id")
    df_test = pd.merge(df_test, create_income_repartition(df_grunnkrets_income_house), how="left", on="grunnkrets_id")

    df_train = pd.merge(df_train, create_hierarchy(df_plaace_hierarchy), how="left", on="plaace_hierarchy_id")
    df_test = pd.merge(df_test, create_hierarchy(df_plaace_hierarchy), how="left", on="plaace_hierarchy_id")

    df_train = pd.merge(df_train, create_grunnkret_geodata(df_grunnkrets_stripped), how="left", on="grunnkrets_id")
    df_test = pd.merge(df_test, create_grunnkret_geodata(df_grunnkrets_stripped), how="left", on="grunnkrets_id")

    df_train = pd.merge(df_train, create_population_age(df_grunnkrets_age_dist), how="left", on="grunnkrets_id")
    df_test = pd.merge(df_test, create_population_age(df_grunnkrets_age_dist), how="left", on="grunnkrets_id")

    df_train["population_density"] = df_train["total_population"] / df_train["area_km2"]
    df_test["population_density"] = df_test["total_population"] / df_test["area_km2"]

    df_train = pd.merge(df_train, find_closest_bus_stop(df_train, df_bus_stops), how="left", on="store_id")
    df_test = pd.merge(df_test, find_closest_bus_stop(df_test, df_bus_stops), how="left", on="store_id")

    df_train['mall_name'] = df_train['mall_name'].fillna('0')
    df_test['mall_name'] = df_test['mall_name'].fillna('0')
    df_train, df_test = encode_feature(df_train, df_test, 'mall_name')

    df_train['chain_name'] = df_train['chain_name'].fillna('0')
    df_test['chain_name'] = df_test['chain_name'].fillna('0')
    df_train, df_test = encode_feature(df_train, df_test, 'chain_name')

    df_train, df_test = encode_feature(df_train, df_test, 'sales_channel_name')

    df_train, df_test = encode_feature(df_train, df_test, 'lv3_desc')
    df_train, df_test = encode_feature(df_train, df_test, 'lv2_desc')
    df_train, df_test = encode_feature(df_train, df_test, 'lv1_desc')

    df_train = fix_municipalities(df_train)
    df_test = fix_municipalities(df_test)
    df_train, df_test = encode_feature(df_train, df_test, 'municipality_name')

    Y_train, df_train = extract_revenue(df_train)

    df_train = drop_non_use_features(df_train)
    df_test = drop_non_use_features(df_test)

    X_train = fill_nan_mean(df_train)
    X_test = fill_nan_mean(df_test)

    X_train, X_test, Y_train = scale_values(X_train, X_test, Y_train)

    return X_train, X_test, Y_train

# Data Building

In [23]:
df_train = pd.read_csv("data/stores_train.csv")
df_test = pd.read_csv("data/stores_test.csv")

# Preprocessing
df_train = fix_outliers(df_train)
df_train = fix_lat_lon(df_train) 
df_test = fix_lat_lon(df_test)

# Features engineering
X_train, X_test, Y_train = features_engineering(df_train, df_test)

In [24]:
features =   [
    'grunnkrets_id',
    'SI_p1',
    'SI_p2', 
    'SI_p3', 
    'SI_all',
    'latxlat',
    'population_density',
    'ih_all_households',
    'BS_closest_mangler_viktighetsnivå',
    'BS_closest_lokalt_knutepunkt',
    'BS_closest_nasjonalt_knutepunkt',
    'BS_closest_regionalt_knutepunkt',
    'BS_closest_annen_viktig_holdeplass',
    'encoded_lv3_desc',
    'encoded_sales_channel_name',
    'encoded_chain_name',
    'encoded_mall_name',
    'encoded_municipality_name',
]

X_train = keep_only_use_features(X_train, features)
X_test = keep_only_use_features(X_test, features)

In [25]:
rf_model = RandomForestRegressor(
    n_estimators=180,
    criterion='squared_error',
    max_depth=None,
    min_samples_split=14,
    min_samples_leaf=11,
    min_weight_fraction_leaf=0.0,
    max_features=None,
    max_leaf_nodes=300,
    min_impurity_decrease=0.0,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    verbose=0,
    warm_start=False,
    ccp_alpha=0.0,
    max_samples=None,
    random_state=SEED,
)

lgbm_model = LGBMRegressor(
    num_leaves=70,
    max_depth=7, 
    n_estimators=2000,
    min_data_in_leaf = 400,
    learning_rate=0.05,
    random_state=SEED,  
)

xgb_model = XGBRegressor(
    objective='reg:squarederror', 
    n_estimators=300, 
    colsample_bytree=0.8958238323555624, 
    gamma=0.11909139052336326,
    learning_rate=0.05983241782780355,
    subsample=0.8889067727422637,
    max_depth=5,
    random_state=SEED,
)

# Model Running

In [26]:
ntrain = X_train.shape[0]
ntest = X_test.shape[0]

NFOLDS = 5 # set number of folds for out-of-fold prediction
kf = KFold(
    n_splits=NFOLDS,
    shuffle=True,
    random_state=SEED
) # K-Folds cross-validator

# oof = out of fold
def get_oof(clf, x_train, y_train, x_test):
    """
    Popular function on Kaggle.
    
    Trains a classifier on 4/5 of the training data and
    predicts the rest (1/5). This procedure is repeated for all 5 folds,
    thus we have predictions for all training set. This prediction is one
    column of meta-data, later on used as a feature column by a meta-algorithm.
    We predict the test part and average predictions across all 3 models.
    
    Keyword arguments:
    clf -- classifier
    x_train -- 4/5 of training data
    y_train -- corresponding labels
    x_test -- all test data
    
    """
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf.split(x_train)):
        x_tr = x_train.iloc[train_index, :]
        y_tr = y_train[train_index]
        x_te = x_train.iloc[test_index, :]

        clf.fit(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [27]:
rf_oof_train, rf_oof_test = get_oof(rf_model, X_train, np.ravel(Y_train), X_test)
lgbm_oof_train, lgbm_oof_test = get_oof(lgbm_model, X_train, np.ravel(Y_train), X_test)
xgb_oof_train, xgb_oof_test = get_oof(xgb_model, X_train, np.ravel(Y_train), X_test)

[LightGBM] [Warning] min_data_in_leaf is set=400, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=400
[LightGBM] [Warning] min_data_in_leaf is set=400, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=400
[LightGBM] [Warning] min_data_in_leaf is set=400, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=400
[LightGBM] [Warning] min_data_in_leaf is set=400, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=400
[LightGBM] [Warning] min_data_in_leaf is set=400, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=400


In [28]:
x_train = np.concatenate((
    rf_oof_train,
    lgbm_oof_train,
    xgb_oof_train,
), axis=1)

x_test = np.concatenate((
    rf_oof_test,
    lgbm_oof_test,
    xgb_oof_test,
), axis=1)

In [29]:
# OOF predictions
meta_df = pd.DataFrame(x_train, columns=[
    'RF',
    'LGBM',
    'XGB',
])
meta_df['label'] = Y_train
meta_df

,RF,LGBM,XGB,label
0,0.839384,1.266116,1.074493,1.278708
1,0.978402,1.433658,1.134399,1.394942
2,0.818390,0.802136,0.928183,1.232971
3,0.714633,0.793416,0.739758,1.012669
4,0.619073,0.957293,0.835196,0.742568
...,...,...,...,...
12854,0.546683,0.387816,0.539882,0.036629
12855,0.906838,0.532016,0.552822,0.449633
12856,0.719494,0.588411,0.743718,1.593563
12857,0.590086,0.623193,0.649426,0.666705


In [30]:
META_MODEL = LinearRegression()
META_MODEL.fit(x_train, Y_train)
Y_Pred = META_MODEL.predict(x_test)

# Submission

In [31]:
submission = pd.DataFrame()
submission['id'] = df_test.store_id 
submission['predicted'] = np.asarray(10 ** Y_Pred - 1)
submission.to_csv('submission.csv', index=False)
submission.head()

,id,predicted
0,914206820-914239427-717245,1.912976
1,916789157-916823770-824309,3.192157
2,913341082-977479363-2948,5.810446
3,889682582-889697172-28720,8.828748
4,997991699-998006945-417222,4.795286
